# Santa 2023 - The Polytope Permutation Puzzle
Your task in this competition is to solve each of the given permutation puzzles in the fewest number of moves. See the Evaluation page for more details.

## Files
### puzzle_info.csv
- puzzle_type - Identifies the type of puzzle. Puzzles of the same type have a common set of moves.
- allowed_moves - Describes the moves allowed in solutions of this puzzle type. Each move represents a permutation given in - array form. You are also allowed to use the inverses of these moves in your puzzle solutions.
### puzzle.csv
- id - A unique identifier for each puzzle.
- puzzle_type - Corresponding to puzzle_info.csv.
- solution_state - An arrangement of "colors" describing the solved state of the puzzle, with a semicolon ; delimiter.
- initial_state - An arrangement of colors describing the initial state of the puzzle, with a semicolon ; delimiter. A solution to a puzzle must transform the initial state to the solved state through a sequential application of the puzzle's allowed_moves.
- num_wildcards - The number of "mistakes" allowed in the final state of a solution.
### sample_submission.csv - A submission file in the correct format.
- id - Corresponding to puzzles.csv.
- moves - An initial, unoptimized solution.


In [12]:
import pandas as pd
import json


puzzle_info = (pd.read_csv('puzzle_info.csv')
               .assign(
                   allowed_moves=lambda df: df.allowed_moves.apply(lambda x: json.loads(x.replace("'", '"'))),
               )
               )
puzzles = pd.read_csv('puzzles.csv')


In [66]:
from dataclasses import dataclass
from typing import Dict, List
from sympy.combinatorics import Permutation
from functools import reduce
import numpy as np

@dataclass
class Puzzle:
    """A permutation puzzle."""

    puzzle_id: str
    puzzle_type: str
    allowed_moves: Dict[str, List[int]]
    solution_state: List[str]
    initial_state: List[str]
    num_wildcards: int





In [42]:

def parse_puzzle(row):
    return Puzzle(
        puzzle_id=row.id,
        puzzle_type=row.puzzle_type,
        allowed_moves=row.allowed_moves,
        solution_state=row.solution_state.split(';'),
        initial_state=row.initial_state.split(';'),
        num_wildcards=row.num_wildcards,
    )

In [44]:
puzzles = puzzles.merge(puzzle_info, on='puzzle_type').apply(parse_puzzle, axis=1)

In [81]:
def log_move_on_puzzle(puzzle: Puzzle, moves: List[str]) -> Puzzle:
    """Apply a move to a puzzle and return the new puzzle state."""
    permutations = [Permutation(puzzle.allowed_moves[move]) for move in moves]
    curr_state = puzzle.initial_state.copy()
    print(f" puzzle starting state: {curr_state}")
    for idx, permutation in enumerate(permutations):
        curr_state = permutation(curr_state)
        print(f" puzzle state after move {moves[idx]}: {curr_state}")

def apply_moves(puzzle: Puzzle, moves: List[str]) -> Puzzle:
    """Apply a permutation to a puzzle and return the new puzzle state."""
    permutations = reduce(lambda a,b: a*b, [Permutation(puzzle.allowed_moves[move]) for move in moves])
    final_state = permutations(puzzle.initial_state)    
    return Puzzle(
        puzzle_id=puzzle.puzzle_id,
        puzzle_type=puzzle.puzzle_type,
        allowed_moves=puzzle.allowed_moves,
        solution_state=puzzle.solution_state,
        initial_state=final_state,
        num_wildcards=puzzle.num_wildcards,
    )

def check_solution(puzzle: Puzzle, final_state: List[str]) -> bool:
    """Check if a puzzle is solved."""
    return np.sum(np.array(final_state) != np.array(puzzle.solution_state)) <= puzzle.num_wildcards

def check_moves_solve_puzzle(puzzle: Puzzle, moves: List[str]) -> bool:
    """Check if a list of moves solves a puzzle."""
    permutations = reduce(lambda a,b: a*b, [Permutation(puzzle.allowed_moves[move]) for move in moves])
    final_state = permutations(puzzle.initial_state)    
    return check_solution(puzzle, final_state)


#==============================================================================
# Test
#==============================================================================
puzzle = puzzles[0]
print(puzzle.puzzle_type)
print(puzzle.initial_state)
print(puzzle.solution_state)
print(puzzle.num_wildcards)
assert check_solution(puzzle, puzzle.solution_state)
assert check_solution(puzzle, puzzle.initial_state) == False

cube_2/2/2
['D', 'E', 'D', 'A', 'E', 'B', 'A', 'B', 'C', 'A', 'C', 'A', 'D', 'C', 'D', 'F', 'F', 'F', 'E', 'E', 'B', 'F', 'B', 'C']
['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F']
0


In [25]:
def solve(puzzle: Puzzle):
    """Solve a puzzle."""
    available_moves = list(puzzle.allowed_moves.keys())
    
    

,id,puzzle_type,solution_state,initial_state,num_wildcards
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0
...,...,...,...,...,...
393,393,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,D;D;L;A;P;E;R;U;U;C;S;R;J;B;E;G;O;J;F;Q;R;E;D;...,0
394,394,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,V;L;N;G;B;V;R;E;H;A;K;S;I;N;G;E;V;C;L;G;S;M;P;...,0
395,395,globe_3/33,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N12;N219;N227;N198;N4;N208;N214;N245;N56;N55;N...,0
396,396,globe_8/25,A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...,V;P;F;L;P;X;O;A;J;b;V;Y;D;Y;C;X;J;F;U;G;d;L;b;...,0
